# TFIDF选取关键词

## 预设

In [1]:
import sys
import os
import openpyxl
import csv
import re
import pandas as pd
import jieba
import jieba.posseg as psg
import jieba.analyse as analyse
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import cn2an
import functools

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

### 全局常量

In [3]:
ExtraDictionaryPath = '../setting/dict.txt'
StopWordPath = 'set/stopwords.txt'
KeywordTopNumber = 10
minLengthOfWord = 3
DataFilePath = '../data/西安城市形象数据_分词.pkl'
IDFFilePath = '../setting/pg_idf.txt'

## 数据读取

In [4]:
data = pd.read_pickle(DataFilePath)

In [5]:
data['分词'].sample(3)

15217                                                      
9322      学校 学校 气候 痘痘 中药 头发 试试 女生节 部门 男孩子 卡片 硬生生 拼音 汉字 老...
145642                                                   奶茶
Name: 分词, dtype: object

## 处理函数工具

In [17]:
# 单篇文档提取关键词
def get_key_word_from_a_doc(content, topK=KeywordTopNumber):
    return ' '.join(analyse.extract_tags(content, topK))

## 数据处理

### 获取总体关键字

In [14]:
corpus = [].append(data['内容'][2])
# tfidf_vect = TfidfVectorizer(use_idf=True, smooth_idf=True, norm=None, max_features=500, max_df = 0.05,
#                                 min_df = 15)
tfidf_vect = TfidfVectorizer(use_idf=True, smooth_idf=True, norm=None, max_features=1500, max_df = 0.5,
                                min_df = 10)
tfidf_features = tfidf_vect.fit_transform(corpus)
keywords = tfidf_vect.get_feature_names()

TypeError: 'NoneType' object is not iterable

#### 获取总体idf表

In [9]:
# idf_df_data = {'word':keywords}
idf_df_data = {'word':keywords,'idf':tfidf_vect.idf_}
idf_df = pd.DataFrame(idf_df_data)
display(idf_df)

,word,idf
0,4s店,7.933063
1,一体,7.600288
2,一家人,7.294906
3,一带,6.721738
4,一楼,7.478398
...,...,...
1495,鼻尖,7.548730
1496,鼻翼,7.487810
1497,鼻部,7.882693
1498,龅牙,6.123022


#### 保存idf表

In [14]:
idf_df.to_csv(IDFFilePath, header=None, index=None, sep=' ', mode='a')


### 获取个体关键字

#### 获取字

In [7]:
analyse.set_idf_path(IDFFilePath);

In [8]:
keywords = analyse.extract_tags(data['内容'][2], 10, withWeight=True)
keywords

[('就是', 0.24088880000000001),
 ('好吃', 0.2159506919416227),
 ('西安', 0.18715726634940635),
 ('真的', 0.18715726634940635),
 ('可以', 0.1439671279610818),
 ('这家', 0.10077698957275726),
 ('但是', 0.10077698957275726),
 ('因为', 0.08638027677664908),
 ('一个', 0.0719835639805409),
 ('现在', 0.05758685118443272)]

In [28]:
for keyword in keywords:
    print(keyword)
    print(idf_df[idf_df['word']==keyword].size)

好吃
0
西安
0
真的
0
可以
0
这家
0
但是
0
因为
0
一个
0
现在
0
就是
0


In [52]:
data['二词关键词'] = data['内容'].apply(get_key_word_from_a_doc)

#### 获取得分

In [38]:
a = analyse.TFIDF(idf_path=IDFFilePath)

In [39]:
a.extract_tags(data['内容'][2],topK= 10, withWeight=True)

[('就是', 0.24088880000000001),
 ('好吃', 0.2159506919416227),
 ('西安', 0.18715726634940635),
 ('真的', 0.18715726634940635),
 ('可以', 0.1439671279610818),
 ('这家', 0.10077698957275726),
 ('但是', 0.10077698957275726),
 ('因为', 0.08638027677664908),
 ('一个', 0.0719835639805409),
 ('现在', 0.05758685118443272)]

## 数据展示&保存

In [16]:
df_to_show = data

In [35]:
display(df_to_show.sample(3)['二词关键词'])

47869    摄影 ## 守风人 凌安 DearWhite 不凡 亲爱 西安
55437                               小康家庭
376       秦岭 葛牌 石船 山沟 龙王庙 沟口 褚家 粉房 沟内 支沟
Name: 二词关键词, dtype: object

### 数据保存

In [18]:
df_to_save = data

In [19]:
new_path = get_new_path(DataFilePath, 'tfidf分词')


In [20]:
data.to_pickle(new_path)